In [1]:
import function_list as ff
import os
import numpy as np
import nibabel as nib 
import math
from scipy import misc
import matplotlib.pyplot as plt
from PIL import Image
import cv2
print('ok')

ok


In [2]:
base_dir = '/Volumes/McVeighLab/projects/Zhennong/AI/CNN/all-classes-all-phases-1.5'
patient_list = ff.find_all_target_files(['ucsd_toshiba/000'],base_dir)

In [3]:
# load plane information
for patient in patient_list:
    patient_id = os.path.basename(patient)
    patient_class = os.path.basename(os.path.dirname(patient))
    print(patient_class,patient_id)


    mpr_planes = nib.load(os.path.join(patient,'mpr-new-nii-sm-1.5/2C/0.nii.gz')) 
    mpr_data = mpr_planes.get_fdata()
    
    seg = nib.load(os.path.join(patient,'seg-nii-sm/0.nii.gz')); seg_data = seg.get_fdata()
    volume = nib.load(os.path.join(patient,'img-nii-sm/0.nii.gz')); volume_data = volume.get_fdata()
    
    # ground truth vectors
    vector_4C = ff.get_ground_truth_vectors(patient,'4C_MR.npy')
    vector_SA = ff.get_ground_truth_vectors(patient,'BASAL_MR.npy')
    image_center = vector_SA['img_center']
    
    # get a list of center points of SA stack, the orientation keeps constant
    n = ff.normalize(np.cross(vector_SA['x'],vector_SA['y']))
    ahead,behind = ff.find_num_of_slices_in_SAX(mpr_data,image_center,vector_SA['t'],vector_SA['x'],vector_SA['y'],seg_data,8)
    center_list = ff.find_center_list_whole_stack_upsampled(image_center+vector_SA['t'],n,ahead,behind,1.5,default_thickness = 8)
    
    # get affine matrix
    volume_affine = ff.check_affine(os.path.join(patient,'img-nii-sm/0.nii.gz'))
    A_4C_gt = ff.get_affine_from_vectors(mpr_data,volume_affine,vector_4C)

ucsd_toshiba 000


## Find surface points

In [12]:
# define a function for surface point search
def surface_point_search(seg,target_val):
    count,p_list = ff.count_pixel(seg,target_val)
    p_list = np.asarray(p_list).reshape((count,3))
    
    s_list = []
    for p in p_list:
        neighbour_no = 0
        for i in [-1,0,1]:
            for j in [-1,0,1]:
                for k in [-1,0,1]:
                    if seg[p[0]+i,p[1]+j,p[-1]+k] == 1:
                        neighbour_no += 1
        if neighbour_no != 27:
            s_list.append(p)
        l = len(s_list)
    return np.asarray(s_list).reshape((l,3))
            

In [13]:
count,p_list = ff.count_pixel(seg_data,1)
s_list = surface_point_search(seg_data,1)
print(count,s_list.shape)

32283 (11772, 3)


In [14]:
new_seg = np.zeros((seg_data.shape))
for p in p_list:
    new_seg[p[0],p[1],p[-1]] = 1
for s in s_list:
    new_seg[s[0],s[1],s[-1]] = 2
new_image = nib.Nifti1Image(new_seg,seg.affine)
save_path = '/Volumes/McVeighLab/projects/Zhennong/Video Prediction/Test_Data/LV'
nib.save(new_image,os.path.join(save_path,'surface.nii'))

## Inflate the LV segmentation

In [39]:
i_list = []
inflate_range = 3
for s in s_list:
    for i in np.linspace(-inflate_range,inflate_range,inflate_range * 2 +1):
        for j in np.linspace(-inflate_range,inflate_range,inflate_range * 2 +1):
            for k in np.linspace(-inflate_range,inflate_range,inflate_range * 2 +1):
                i_list.append([s[0]+i,s[1]+j,s[-1]+k])
        

In [40]:
i_list = np.asarray(i_list).reshape((len(i_list),3))
i_list = np.unique(i_list,axis = 0)
print(i_list.shape)

(64416, 3)


In [41]:
new_seg2 = np.zeros((seg_data.shape))
for i in i_list:
    new_seg2[int(i[0]),int(i[1]),int(i[-1])] = 3
for p in p_list:
    new_seg2[p[0],p[1],p[-1]] = 1
for s in s_list:
    new_seg2[s[0],s[1],s[-1]] = 2
new_image2 = nib.Nifti1Image(new_seg2,seg.affine)
nib.save(new_image2,os.path.join(save_path,'inflate.nii'))